In [1]:
using Compat, Random, Distributions, Plots, LinearAlgebra, Statistics, LaTeXStrings, SparseArrays, CPUTime, JLD2
using SpecialFunctions, GSL, SuiteSparse, Distributed, Roots, NBInclude
gr(fmt=:png)
e = 2.7182818284
Random.seed!(1);

In [2]:
@nbinclude("setup.ipynb")
using .setup

In [3]:
function fill_matrix(Λ,par_grid)
    K = par_grid.K
    matr = zeros(K,K)
    matr[CartesianIndex.(2:K,2:K)] = Λ
    matr[CartesianIndex.(1:K-1,2:K)] = ones(K-1) - Λ
    matr[:,1] = ones(K) - sum(matr[:,2:end],dims=2)
    return matr
end

function fill_productivity(par_grid,l_mean,l_var,λ_bar,Δλ,display)
    Λ, par_grid.labor = ones(par_grid.K), ones(par_grid.K)
    Λ[end] = λ_bar
    l_this = []
    for j in 0.05:0.1:0.95
        append!(l_this,find_zero(x->erf(x/sqrt(l_var))/2+0.5-j,0.0))
    end
    par_grid.labor = exp.(l_this)
    par_grid.labor = par_grid.labor/mean(par_grid.labor)*l_mean
    par_grid.productivity = kron(par_grid.labor,ones(par_grid.N))
    for i in 1:par_grid.K-1
            Λ[end-i] = max(0,λ_bar - Δλ*i)
    end
    Λ = Λ[2:end]

    par_grid.trans_matrix = fill_matrix(Λ,par_grid) - Matrix(I,par_grid.K,par_grid.K)
    eig = eigen(Matrix(Diagonal(ones(par_grid.K))-par_grid.trans_matrix'))
    number = findfirst(z->(Real(z*conj(z))<1.0001)&&(Real(z*conj(z))>0.9999),eig.values)
    income_distr = Real.(eig.vectors[:,number])
    income_distr = income_distr/sum(income_distr)
    if display
        print("probability to switch decile: ",round(100*income_distr'*(ones(par_grid.K)-exp.(diag(par_grid.trans_matrix)*12)),digits=2),"% per year, ",
            round(100*income_distr'*(ones(par_grid.K)-exp.(diag(par_grid.trans_matrix)*6)),digits=2),"% per semester, ",
            round(100*income_distr'*(ones(par_grid.K)-exp.(diag(par_grid.trans_matrix)*3)),digits=2),"% per quarter, ",
            round(100*income_distr'*(ones(par_grid.K)-exp.(diag(par_grid.trans_matrix))),digits=2),"% per month")
    end
    plot_prod = plot(par_grid.labor,income_distr,color=:blue,label="distribution",seriestype=:scatter,markersize=10,legend=:false,tickfontsize=15,ylim=[0.04,0.13])
    return par_grid, plot_prod
end

fill_productivity (generic function with 1 method)

In [4]:
ρ_y, σ_p, σ_t = 0.963, sqrt(0.021), sqrt(0.196) # from Hong (2023)
l_var = σ_p^2/(1-ρ_y^2) + σ_t^2 
l_mean = 0.09472044076948055 # mean log productivity (set in calibration)
λ_bar = 0.982509946100006 # probability of staying at the top (set in calibration)
Δλ = 0.034996393288546944 # step in staying probabilities (set in calibration)
K = 10 # number of productivity levels (choice)
labor = [] # productivity grid (to be filled below)
productivity = [] # stacked productivity grid (to be filled below)
trans_matrix = [] # transition matrix (to be filled below)
N = 1501 # number of asset levels (choice)
da = 0.01 # asset grid step (choice)
a_bar = -0.25 # hard borrowing constraint (choice)
q_grid = collect(a_bar:da:a_bar + da*(N-1)) # asset grid
assets = kron(ones(K),q_grid) # stacked asset grid

par_grid = grid_parameters(l_mean,l_var,Δλ,λ_bar,K,labor,productivity,trans_matrix,N,da,a_bar,q_grid,assets) # initialize the grid structure

par_grid, plot_prod = fill_productivity(par_grid,l_mean,l_var,λ_bar,Δλ,true); # fill the labor grid and transition matrix

probability to switch decile: 76.62% per year, 57.97% per semester, 37.71% per quarter, 15.37% per month

In [5]:
χ1 = 0.8 # normalization to generate l=1 when u'(c)=1, w=1, and τ=0.2
χ2 = 0.5 # elasticity of labor supply from Kaplan Moll Violante (2018)
ρ = 0.011611134781250509 # discount rate (set in calibration)
σ = 1.0745164406470842 # IES (set in calibration)
τ_w = 0.2 # tax rate (set in calibration)
    
ζ_N = 0.5397226696266482 # share of workers in non-tradables (set in calibration)
ζ_T = 1 - ζ_N # share of workers in tradables (set in calibration)
    
r_d = 0.004166666666666667 # interest rate on deposits, 5% annually (choice)
r_l = 0.018394789900282607 # interest rate on loans (set in calibration)
    
κ = 0.0067/ρ # slope of Phillips curve from Alves (2020)
θ = 1.5 # elasticity of substitution between tradables and non-tradables (choice)
η = 0.5409765755720075 # weight on tradables (set in calibration)
α = 0.6108521207850077 # weight on domestic goods in tradables (set in calibration)
θ_g = 3.0 # elasticity of substitution between domestic and foreign tradables (choice)
θ_e = 3.0 # elasticity of export demand (choice)

par = model_parameters(χ1,χ2,ρ,σ,τ_w,ζ_N,ζ_T,r_d,r_l,κ,θ,η,α,θ_g,θ_e); # fill model parameters

In [6]:
prod_N = 0.6 # productivity in non-tradables relative to tradables
prod_T = 1.0 # productivity in tradables (could be changed to other numbers if prod_N changes)
export_share = 0.214 # export share
NT_to_GDP = 0.457 # share of non-tradables in GDP
foreign_to_gdp = 0.08816214251 # foreign ownership of government debt
MPC_target = [0.28, 0.183, 0.188, 0.213, 0.237, 0.219, 0.239, 0.183, 0.197, 0.106] # from Hong (2023)

@save "parameters.jld2" par par_grid
@save "external_parameters.jld2" prod_N prod_T export_share NT_to_GDP foreign_to_gdp MPC_target

Parameters for alternative calibrations:

* equal productivities across sectors: *prod_N=prod_T=0.8*
* productivity levels switched: *prod_N=1.0*, *prod_T=0.6*

In [7]:
prod_N = 0.8 # productivity in non-tradables relative to tradables
prod_T = 0.8 # productivity in tradables (could be changed to other numbers if prod_N changes)
export_share = 0.214 # export share
NT_to_GDP = 0.457 # share of non-tradables in GDP
foreign_to_gdp = 0.08816214251 # foreign ownership of government debt
MPC_target = [0.28, 0.183, 0.188, 0.213, 0.237, 0.219, 0.239, 0.183, 0.197, 0.106] # from Hong (2023)

@save "external_parameters_equal.jld2" prod_N prod_T export_share NT_to_GDP foreign_to_gdp MPC_target

In [8]:
prod_N = 1.0 # productivity in non-tradables relative to tradables
prod_T = 0.6 # productivity in tradables (could be changed to other numbers if prod_N changes)
export_share = 0.214 # export share
NT_to_GDP = 0.457 # share of non-tradables in GDP
foreign_to_gdp = 0.08816214251 # foreign ownership of government debt
MPC_target = [0.28, 0.183, 0.188, 0.213, 0.237, 0.219, 0.239, 0.183, 0.197, 0.106] # from Hong (2023)

@save "external_parameters_switched.jld2" prod_N prod_T export_share NT_to_GDP foreign_to_gdp MPC_target